Import packages

In [1]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
import re
import json
import geoip2.database
from datetime import datetime
import time
from transformers import AutoTokenizer, RobertaForSequenceClassification, AutoModelForTokenClassification, AutoModelForSequenceClassification
import torch

In [2]:
# with open("mongodb_comments.json",mode="r",errors= "ignore") as file:
#     Data = json.load(file)
# data = pd.read_json("mongodb_comments.json", encoding= 'utf-8')
# data.head(5)

In [2]:
#function for retrieving data from MongoDB
def get_data(n_records):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['Sentiment_Analysis']
    collection = db['FPTPlay_Data']
    cursor = collection.find().limit(n_records)
    data = pd.DataFrame(list(cursor))
    return data

In [3]:
data = get_data(100000)

Delete unnecessary columns

In [5]:
data.drop(columns=["_id","ranking", "layer", "object_id", "report_count", "report_reason", "dislike", "device_id"], inplace=True)
data.head(5)

,user_id,like,user_email,comment_on,ip,user_fullname,publish_status,content,reviewer,review_status,timestamp,device,comment_status
0,464817,12,13@fpt.com.vn,event,42.116.8.164,tuyen tuyen,1,chuong trinh hay qua,vuluc88,1,1431513137,android,0
1,28,4,lokiheero@gmail.com,vod,42.116.8.164,Quốc Trung,1,hehe không biết chừng nào có tập 6,vuluc88,1,1431513337,android,0
2,424467,1,n_x_cuong@yahoo.com,vod,None,nguyen xuan cuong,1,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",vuluc88,1,1431525776,web-playfpt,0
3,3839,4,xxx@gmail.com,vod,42.116.8.164,xxxxxurruyu,1,phim hay qua,vuluc88,1,1431566632,android,0
4,527201,5,cloudytang@gmail.com,vod,42.116.8.164,Đàm Ngọc Vân,1,hay qua do,vuluc88,1,1431573623,android,0


Generate new column called "province" that identifies which province user is lives in based on ip adress

In [6]:
database_path = r'E:\hienntt\github\PROJECT\UNIACE_project\GeoLite2-City.mmdb'
def get_province(ip):
    with geoip2.database.Reader(database_path) as reader:
        try:
            response = reader.city(ip)
            province = response.subdivisions.most_specific.name
            return province
        except:
            return "Invalid IP"
data["province"] = data["ip"].apply(lambda x: get_province(x) if x != np.nan else None)
data["province"].unique()

array(['Ho Chi Minh', 'Invalid IP', 'Đồng Nai Province',
       'Lâm Đồng Province', None, 'Thái Bình Province', 'Hanoi',
       'Can Tho', 'Tây Ninh Province', 'Bến Tre Province', 'Da Nang',
       'Nghệ An Province', 'Quảng Bình Province', 'Lào Cai Province',
       'Bắc Ninh Province', 'Bình Định Province', 'Hải Dương Province',
       'Quảng Nam Province', 'Haiphong', 'Thái Nguyên Province',
       'Nam Định Province', 'Vĩnh Phúc Province', 'Đắk Lắk',
       'Bình Phước Province', 'Bình Dương Province', 'Long An Povince',
       'Khánh Hòa Province', 'Bắc Giang Province', 'Hòa Bình Province',
       'Quảng Ninh', 'Trà Vinh Province', 'Hưng Yên Province',
       'Bà Rịa–Vũng Tàu Province', 'Thừa Thiên Huế Province',
       'Quảng Ngãi Province', 'Thanh Hóa Province', 'Vĩnh Long Province',
       'Bạc Liêu Province', 'An Giang Province', 'Hà Nam Province',
       'Kiên Giang Province', 'Đồng Tháp Province', 'Tiền Giang',
       'Hậu Giang', 'Cà Mau Province', 'Kon Tum', 'Tuyên Quang 

Convert "timestamp" column to datetime

In [7]:
data["date_time"] = pd.to_datetime(data["timestamp"], unit= "s")
data["date_time"]

0       2015-05-13 10:32:17
1       2015-05-13 10:35:37
2       2015-05-13 14:02:56
3       2015-05-14 01:23:52
4       2015-05-14 03:20:23
                ...        
99995   2016-05-23 10:10:15
99996   2016-05-23 10:10:59
99997   2016-05-23 10:24:57
99998   2016-05-23 11:06:44
99999   2016-05-23 11:13:04
Name: date_time, Length: 100000, dtype: datetime64[ns]

In [8]:
data.drop("timestamp", axis= 1, inplace= True)

Because my device's configuration is weak, I only get 100,000  records for analysis

In [9]:
new_data = data[(data["province"] != "Invalid IP") & (data["province"].notnull())][0:30000]
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 0 to 43509
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         30000 non-null  int64         
 1   like            30000 non-null  int64         
 2   user_email      30000 non-null  object        
 3   comment_on      30000 non-null  object        
 4   ip              30000 non-null  object        
 5   user_fullname   30000 non-null  object        
 6   publish_status  30000 non-null  int64         
 7   content         30000 non-null  object        
 8   reviewer        30000 non-null  object        
 9   review_status   30000 non-null  int64         
 10  device          30000 non-null  object        
 11  comment_status  30000 non-null  int64         
 12  province        30000 non-null  object        
 13  date_time       30000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(8)
memory usage: 3.4

Implement Sentiment analysis for "content" column.

In [10]:
# Firstly, ensure the 'content' column is of type str
new_data['content'] = new_data['content'].astype(str) 

In [11]:
#Next, I do some basic text cleaning steps for improving performance.
#Function for basic text normalization
def normalize_content(content):
    content = content.lower()  # Lowercase
    content = re.sub(r'\s+', ' ', content)  # Replace multiple spaces with a single space
    content = re.sub(r'[^\w\s]', '', content)  # Remove punctuation
    return content.strip()

In [12]:
#Normalize the 'content' column
new_data["content"] = new_data["content"].apply(normalize_content)

In [13]:
max_lengths = new_data.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).max()
print(max_lengths)

C:\Users\hungl\AppData\Local\Temp\ipykernel_29232\3767677497.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_lengths = new_data.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).max()


user_id             6
like                3
user_email         45
comment_on          7
ip                 15
user_fullname      53
publish_status      1
content           878
reviewer           14
review_status       1
device              8
comment_status      1
province           24
date_time          19
dtype: int64


As data is in Vietnamese, I will use the PhoBERT model for sentiment analysis. 
The PhoBERT model is a RoBERTa-based model that is pre-trained on a large corpus of Vietnamese text. It is available in the transformers library. 
PhoBERT is a strong model for Vietnamese text, but it was trained on text with full diacritics. Then it is well-suited for Vietnamese text with accents. For text without accents, I pre-process the data to match the model's input requirements with Transformer model for inserting Vietnamese accent marks. This model inserts accent marks (diacritics) for Vietnamese texts that don't have them (or texts with some words accented and some not).
Link of Transformer model for inserting Vietnamese accent marks. https://huggingface.co/peterhung/vietnamese-accent-marker-xlm-roberta

In [14]:
from tqdm.notebook import tqdm
# Load Accent Marker Model
def load_accent_marker_model():
    model_path = "peterhung/vietnamese-accent-marker-xlm-roberta"
    tokenizer = AutoTokenizer.from_pretrained(model_path, add_prefix_space=True)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    return model, tokenizer

accent_model, accent_tokenizer = load_accent_marker_model()

# Load Sentiment Analysis Model
def load_sentiment_model():
    model_path = "wonrax/phobert-base-vietnamese-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    return model, tokenizer

sentiment_model, sentiment_tokenizer = load_sentiment_model()

# Set device for faster processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accent_model.to(device)
sentiment_model.to(device)
accent_model.eval()
sentiment_model.eval()

# Load the tags set file for accenting
def _load_tags_set(fpath):
    labels = []
    with open(fpath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                labels.append(line)
    return labels

label_list = _load_tags_set(r'C:\Users\hungl\Desktop\ThuHien\selected_tags_names.txt')

# Supporting functions for merging tokens and applying accents
TOKENIZER_WORD_PREFIX = "▁"

def merge_tokens_and_preds(tokens, predictions): 
    merged_tokens_preds = []
    i = 0
    while i < len(tokens):
        tok = tokens[i]
        label_indexes = set([predictions[i]])
        if tok.startswith(TOKENIZER_WORD_PREFIX):
            tok_no_prefix = tok[len(TOKENIZER_WORD_PREFIX):]
            cur_word_toks = [tok_no_prefix]
            j = i + 1
            while j < len(tokens):
                if not tokens[j].startswith(TOKENIZER_WORD_PREFIX):
                    cur_word_toks.append(tokens[j])
                    label_indexes.add(predictions[j])
                    j += 1
                else:
                    break
            cur_word = ''.join(cur_word_toks)
            merged_tokens_preds.append((cur_word, label_indexes))
            i = j
        else:
            merged_tokens_preds.append((tok, label_indexes))
            i += 1
    return merged_tokens_preds

def get_accented_words(merged_tokens_preds, label_list):
    accented_words = []
    for word_raw, label_indexes in merged_tokens_preds:
        for label_index in label_indexes:
            tag_name = label_list[int(label_index)]
            raw, vowel = tag_name.split("-")
            if raw and raw in word_raw:
                word_accented = word_raw.replace(raw, vowel)
                break
        else:
            word_accented = word_raw
        accented_words.append(word_accented)
    return accented_words

def insert_accents(text, model, tokenizer, label_list):
    our_tokens = text.strip().split()
    inputs = tokenizer(our_tokens, is_split_into_words=True, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])[1:-1]  # exclude <s> and </s>
    
    with torch.no_grad():
        inputs.to(device)
        outputs = model(**inputs)

    predictions = np.argmax(outputs["logits"].cpu().numpy(), axis=2)[0][1:-1]
    merged_tokens_preds = merge_tokens_and_preds(tokens, predictions)
    accented_words = get_accented_words(merged_tokens_preds, label_list)
    
    return ' '.join(accented_words)

def sentiment_analysis(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=None)
    with torch.no_grad():
        outputs = model(**inputs.to(device))
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return torch.argmax(probs, dim=-1).item()

def process_data(df, batch_size=1000):
    # Initialize columns
    df["accented_content"] = ""
    df["sentiment"] = ""

    accented_comments = []
    sentiments = []
    total_batches = (len(df) + batch_size - 1) // batch_size  # Calculate number of batches

    for start in tqdm(range(0, len(df), batch_size), desc="Processing", total=total_batches):
        end = min(start + batch_size, len(df))
        batch = df["content"][start:end]

        # Process the batch
        batch_accented_comments = [insert_accents(comment, accent_model, accent_tokenizer, label_list) for comment in batch]
        batch_sentiments = [sentiment_analysis(comment, sentiment_model, sentiment_tokenizer) for comment in batch_accented_comments]

        accented_comments.extend(batch_accented_comments)
        sentiments.extend(batch_sentiments)
        
    # Store the results in the DataFrame
    df["accented_content"] = accented_comments
    df["sentiment"] = sentiments

    return df

# Run the processing
processed_data = process_data(new_data)

new_data.to_csv("FPTPlay_Sentiment.csv", index=False, encoding='utf-8-sig')

Processing:   0%|          | 0/30 [00:00<?, ?it/s]

In [16]:
#Change type data to string and convert it with "utf-8" to ensure the font is not wrong
new_data = new_data.astype(str).apply(lambda x: x.str.encode('utf-8').str.decode('utf-8'))

In [17]:
#Make sure the column name is not the same as special word in SQL sever
new_data = new_data.rename(columns={"like":"likes"})
new_data.columns 

Index(['user_id', 'likes', 'user_email', 'comment_on', 'ip', 'user_fullname',
       'publish_status', 'content', 'reviewer', 'review_status', 'device',
       'comment_status', 'province', 'date_time', 'accented_content',
       'sentiment'],
      dtype='object')

Load data into SQL database

In [18]:
import sqlalchemy, pyodbc
from sqlalchemy import create_engine

# Your existing connection string and connection
SERVER = "."
DATABASE = "Sentiment_Analysis"
connection_string = f'DRIVER={{ODBC Driver 13 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)

# Create SQLAlchemy engine using the pyodbc connection
connection_url = sqlalchemy.engine.URL.create(
    "mssql+pyodbc",
    query={"odbc_connect": connection_string}
)
engine = create_engine(connection_url)

new_data.to_sql("FPTPlay_Sentiment", engine, if_exists='append', index=False, dtype={
    'user_id': sqlalchemy.types.NVARCHAR(length=50),
    'likes': sqlalchemy.types.NVARCHAR(length=30),
    'user_email': sqlalchemy.types.NVARCHAR(length=200),
    'comment_on': sqlalchemy.types.NVARCHAR(length=100),
    'ip': sqlalchemy.types.NVARCHAR(length=100),
    'user_fullname': sqlalchemy.types.NVARCHAR(length=200),
    'publish_status': sqlalchemy.types.NVARCHAR(length=30),
    'content': sqlalchemy.types.NVARCHAR(length=1000),
    'reviewer': sqlalchemy.types.NVARCHAR(length=100),
    'review_status': sqlalchemy.types.NVARCHAR(length=30),
    'device': sqlalchemy.types.NVARCHAR(length=100),
    'comment_status': sqlalchemy.types.NVARCHAR(length=30),
    'province': sqlalchemy.types.NVARCHAR(length=100),
    'date_time': sqlalchemy.types.NVARCHAR(length=100),
    'accented_content': sqlalchemy.types.NVARCHAR(length=1000),
    'sentiment': sqlalchemy.types.NVARCHAR(length=30)
})

1

In [19]:
df = pd.read_sql("SELECT * FROM FPTPlay_Sentiment", con = engine)
print(df.head(5))

  user_id likes            user_email comment_on            ip user_fullname  \
0  464817    12         13@fpt.com.vn      event  42.116.8.164   tuyen tuyen   
1      28     4   lokiheero@gmail.com        vod  42.116.8.164    Quốc Trung   
2    3839     4         xxx@gmail.com        vod  42.116.8.164   xxxxxurruyu   
3  527201     5  cloudytang@gmail.com        vod  42.116.8.164  Đàm Ngọc Vân   
4  527201     0  cloudytang@gmail.com        vod  42.116.8.164  Đàm Ngọc Vân   

  publish_status                             content reviewer review_status  \
0              1                chuong trinh hay qua  vuluc88             1   
1              1  hehe không biết chừng nào có tập 6  vuluc88             1   
2              1                        phim hay qua  vuluc88             1   
3              1                          hay qua do  vuluc88             1   
4              0                         comment cai  vuluc88             1   

    device comment_status     province      

In [20]:
conn.close() #Release resources

Check data loaded into SQL server again